# CliffordSim Simple Examples

In this notebook we shall simulate some simple Clifford circuits using CliffordSim.
We will also demonstrate the propagation of Pauli frames using the simulator.


## Bell Pair Circuit

In [1]:
from loom.cliffordsim import Engine
from loom.cliffordsim.operations import (
    Hadamard,
    CNOT,
    Measurement,
    Reset,
    Z,
    Phase,
    PhaseInv,
)
import pprint

In [ ]:
# Bell Pair
operation_list_bell_pair = [Hadamard(0), CNOT(0, 1), Measurement(0), Measurement(1)]
nqubits = 2
seed = None  # Optional seed for reproducibility

cliffordsim_engine = Engine(operation_list_bell_pair, nqubits, seed)

In [3]:
# Runs the Simulator
cliffordsim_engine.run()

In this example, we measure qubits 0 and 1 after applying the Hadamard and CNOT gates. 

CliffordSim performs the measurement and stores the result in its `DataStore` object, accessible through the `data_store` attribute.

In [4]:
pprint.pprint(cliffordsim_engine.data_store.measurements)

{'2': {'d24f42b1-9d10-496e-ae01-32aa623566db': {'flip_results': {},
                                                'is_random': np.True_,
                                                'measurement_result': 1}},
 '3': {'c02eefd4-0c12-4c42-b2c7-fa2e0aac67c2': {'flip_results': {},
                                                'is_random': np.False_,
                                                'measurement_result': np.int8(1)}},
 'time_step': [2, 3]}


The structure of the `DataStore.measurements` can be understood as follows:
1. `time_step` are the parallelized time steps in which the measurement operations were performed.
2. String-fied versions of the values in the `time_step` list as keys. The values for these keys will contain the measurement data from that particular time step.
3. Measurement data, from one `Measurement` Operation is stored in the following format:
```
'uuid of measurement operation': {'is_random': boolean, 'measurement_result': int}
```

In [5]:
print(cliffordsim_engine.stabilizer_set)

{'+ZZ', '-Z_'}


## Five-Qubit Code Initialization

Encoding circuit taken from [here](https://www.researchgate.net/figure/Encoding-circuit-for-the-five-qubit-code-a-Circuit-to-encode-the-logical-minus-state_fig1_337273308)

In [ ]:
operation_list_five_qubit = [
    # Layer 0
    Z(0),
    Hadamard(2),
    Hadamard(3),
    # Layer 1
    PhaseInv(0),
    CNOT(2, 4),
    # Layer 2
    CNOT(3, 1),
    # Layer 3
    Hadamard(1),
    CNOT(3, 4),
    # Layer 4
    CNOT(1, 0),
    PhaseInv(2),
    Phase(3),
    PhaseInv(4),
    # Layer 5
    Phase(0),
    Phase(1),
    Z(2),
    # Layer 6
    CNOT(4, 0),
    # Layer 7
    Hadamard(4),
    # Layer 8
    CNOT(4, 1),
]
nqubits = 5
seed = None

cliffordsim_engine = Engine(operation_list_five_qubit, nqubits, seed)

In [7]:
# Runs the Simulator
cliffordsim_engine.run()

In [8]:
pprint.pprint(cliffordsim_engine.data_store.measurements)

{'time_step': []}


Since there are no measurement operations in this circuit, the measurements attribute of the `DataStore` Object is empty.

In [9]:
print(cliffordsim_engine.stabilizer_set)

{'+XXY_Y', '-ZY__Y', '+XY_YX', '+_XZZX', '+YZ_ZY'}


```{note}
The above may not look exactly how we would expect the state to look like for the 
five-qubit code. Multiplying in pairs appropriately the above Paulis, we can get the
standard set:

- `+XZZX_`
- `+_XZZX`
- `+X_XZZ`
- `+ZX_XZ`
- `+ZZZZZ` (Z Logical operator)
```

## PauliFrame basic example

Below follows a demonstration on how to create and propagate Pauli frames using
a simple circuit that is based on the Repetition code syndrome extraction circuit.

In [ ]:
from loom.cliffordsim.pauli_frame import PauliFrame
from loom.cliffordsim.operations import CreatePauliFrame, RecordPauliFrame

pframe_0 = PauliFrame.from_string("XIIII")
pframe_1 = PauliFrame.from_string("IXIII")

rep_code_pframe_operation_list = [
    CreatePauliFrame(pframe_0),
    CNOT(0, 3),
    CNOT(1, 3),
    CNOT(1, 4),
    CNOT(2, 4),
    Measurement(3),
    Measurement(4),
    Reset(3),
    Reset(4),
    RecordPauliFrame(pframe_0),
    CreatePauliFrame(pframe_1),
    CNOT(0, 3),
    CNOT(1, 3),
    CNOT(1, 4),
    CNOT(2, 4),
    Measurement(3),
    Measurement(4),
    RecordPauliFrame(pframe_1),
]

# Define the Engine
p_frame_engine = Engine(rep_code_pframe_operation_list, 5)
# Run the simulator
p_frame_engine.run()

In [ ]:
# Show propagated pauli frames
p_frame_engine.data_store.get_pframes()

({'initial_pauli_frame': PauliFrame: X____,
  'recorded_pauli_frame': PauliFrame: X__X_},
 {'initial_pauli_frame': PauliFrame: _X___,
  'recorded_pauli_frame': PauliFrame: _X_XX})